In [1]:

# load the packages needed
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from src.solver import Solver
from src.model import AR, MA
from src.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
import src.data_processor as dp
import src.inference 
from src.ts_gen import ar1_gen
data = np.loadtxt("../data/SP500array.csv", delimiter=',')

In [2]:
X=np.array([data[0:100,0]])
Y=dp.get_return(X)
X = ar1_gen(0.5, sigma = 1, time = 200, num = 1, burnin = 2000)

In [3]:
lag = 1
sigma = 1.0
intercept = 0.1
phi = np.random.randn(1, 1)
AR_model = AR(lag=lag, phi=phi, sigma=sigma, intercept=intercept)
AR_model.params

{'intercept': 0.1, 'phi': array([[ 1.24430269]]), 'sigma': 1.0}

In [4]:
_, grads = AR_model.loss(Y)
solver = Solver(AR_model, Y,
                          update_rule='sgd',
                          optim_config={
                            'learning_rate': 1e-5,
                          },
                          
                          num_epochs=1000, batch_size=1,
                          print_every=10)
solver.train()

1000
the loss is %f [ 89.70377951]
the loss is %f [ 88.72760013]
the loss is %f [ 87.73159544]
the loss is %f [ 86.71494382]
the loss is %f [ 85.67677154]
the loss is %f [ 84.6161482]
the loss is %f [ 83.53208178]
the loss is %f [ 82.423513]
the loss is %f [ 81.2893092]
the loss is %f [ 80.12825731]
the loss is %f [ 78.93905611]
the loss is %f [ 77.72030748]
the loss is %f [ 76.47050649]
the loss is %f [ 75.18803029]
the loss is %f [ 73.87112545]
the loss is %f [ 72.51789354]
the loss is %f [ 71.12627475]
the loss is %f [ 69.694029]
the loss is %f [ 68.21871432]
the loss is %f [ 66.69766187]
the loss is %f [ 65.12794687]
the loss is %f [ 63.5063549]
the loss is %f [ 61.82934228]
the loss is %f [ 60.09298956]
the loss is %f [ 58.29294647]
the loss is %f [ 56.42436639]
the loss is %f [ 54.48182791]
the loss is %f [ 52.45924016]
the loss is %f [ 50.34972797]
the loss is %f [ 48.14549123]
the loss is %f [ 45.83763118]
the loss is %f [ 43.41593408]
the loss is %f [ 40.8685988]
the loss is %

In [5]:
AR_model.params

{'intercept': array([-0.00011566]),
 'phi': array([[ 1.20430545]]),
 'sigma': array([ 0.19317783])}

In [9]:
AR_model.predict(X,10)

array([[ -2.89425513e+01,   3.26105967e+02,  -4.10108895e+03,
          5.11027883e+04,  -6.37249119e+05,   7.94599488e+06,
         -9.90807697e+07,   1.23546458e+09,  -1.54053382e+10,
          1.92093281e+11]])

In [8]:
from src.ts_gen import ar1_gen
import src.inference as inf

x = ar1_gen(0.5, sigma = 1, time = 200, num = 1, burnin = 2000)
X=np.array([data[0:100,0]])
Y=dp.get_return(X)

y = inf.yule_walker(x, order =1, method = "mle")
phi,sigma = inf.yule_walker(Y, order =3, method = "mle")
print phi 
print sigma



[-0.19558968 -0.04886918  0.04002632]
0.000119593894429


In [9]:

phi

array([-0.19558968, -0.04886918,  0.04002632])

In [10]:
AR_model = AR(lag=3, phi=phi, sigma=sigma, intercept=0)

In [11]:
rt=AR_model.predict(Y,10)
rt

array([[ -1.47296356e-03,  -4.76342358e-04,   5.03069047e-04,
         -1.34073966e-04,  -1.74273183e-05,   3.00966910e-05,
         -1.04014310e-05,  -1.33939402e-07,   1.73916635e-06,
         -7.49948493e-07]])

In [13]:
z=dp.get_price(X[0,99],rt)
z

array([[ 47.23232588,  47.20982712,  47.23357692,  47.22724413,
         47.22642109,  47.22784244,  47.22735121,  47.22734488,
         47.22742702,  47.2273916 ]])

In [15]:
X.shape[0]

1

In [18]:
data[100:110,0]

array([ 47.543,  46.906,  47.235,  47.235,  47.157,  46.984,  46.762,
        46.752,  46.501,  46.81 ])

In [17]:
import src.trading as tra
X[0,0:100].shape[0]

100

In [18]:
tra.trade(np.array([X[0,0:100]]),z,AR_model,4,4,100)

array([[ 100.       ,  100.       ,  100.0503069,  100.0503069]])